In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0.1, 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )


In [13]:
from langchain.prompts import FewShotChatMessagePromptTemplate


examples = [
    {
        "question": "500 Days of Summer",
        "answer": "🌞🌧️💔"
    },
    {
        "question": "Life of Pi",
        "answer": "🐯🌊🚣"
    },
    {
        "question": "Inside Out",
        "answer": "😄😢😡"
    }
]


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI bot that represents movies using exactly three emojis."),
    ("human", "Can you show me the emojis for 'The Godfather'?"),
    ("ai", "👨‍👨‍👦🔫🍝"),
    ("human", "What about 'Top Gun'?"),
    ("ai", "🛩️👨‍✈️🔥"),
    ("human", "{movie_title}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=prompt,
    input_variables=["movie_title"],
)

formatted_prompt = prompt.format(movie_title="Harry Potter")
print(formatted_prompt)

System: You are an AI bot that represents movies using exactly three emojis.
Human: Can you show me the emojis for 'The Godfather'?
AI: 👨‍👨‍👦🔫🍝
Human: What about 'Top Gun'?
AI: 🛩️👨‍✈️🔥
Human: Harry Potter


In [19]:
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough

memory = ConversationBufferMemory(llm=chat)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | chat


def invoke_chain(movie_title):
    result = chain.invoke({"movie_title": movie_title})
    memory.save_context(
        {"input": movie_title},
        {"output": result.content},
    )
    print(result)

In [20]:

invoke_chain("Harry Potter")

⚡🧙🏰content='⚡🧙🏰'


In [21]:
invoke_chain("The Spider Man")

🕷️🕸️🦸‍♂️content='🕷️🕸️🦸\u200d♂️'


In [22]:
def get_prev_movie_question():
    history = memory.load_memory_variables({})["history"]

    lines = history.split("\n")
    for line in reversed(lines): 
        if line.startswith("Human:"):
            return line.replace("Human:", "").strip()  
    return "No movie has been asked yet."

prev_movie = get_prev_movie_question()
print("The prev movie you asked about was:", prev_movie)

The prev movie you asked about was: The Spider Man
